In [1]:
import numpy as np
import matplotlib as plt

# Square layout, 2-component R-D system with fixed stoich

A reaction-diffusion model is a mathematical model which calculates the concentration of two substances at a given time based on the substances'

- diffusion
- feed rate (how fast the substances are being created by the reaction OR a source)
- removal rate (how fast the substances are being used up by the reaction OR a sink)
- type of reaction between the two

For the reaction $u + 2v \rightarrow 3v$ (i.e. two of species V come together to convert a single U into V) this is governed by the differential equations

\begin{equation}
\frac{\partial u}{\partial t} = r_u \nabla^2 u - uv^2 + f(1-u)
\end{equation}

\begin{equation}
\frac{\partial v}{\partial t} = r_v \nabla^2 v + uv^2 - (f+k)v
\end{equation}

Where $u$ and $v$ are the concentrations of the two reactants, f is the rate at which reactant u is fed into the system (by an outside source), k is the rate at which reactant v is removed from the system (by an outside sink), and $r_u$ and $r_v$ are the diffusion rates of $u$ and $v$ respectively.

The system after a finite time step $dt$ is therefore:

\begin{equation}
u_2 = u_1 + [r_u \nabla^2 u - uv^2 + f(1-u)]dt
\end{equation}

\begin{equation}
v_2 = v_1 + [r_v \nabla^2 v + uv^2 - (f+k)v]dt.
\end{equation}

In [ ]:
# need an input grid of values (initial conditions)